In [1]:
from keras.applications import VGG16 

img_rows = 224
img_cols = 224

model = VGG16( weights = 'imagenet', 
               include_top = False, 
               input_shape = (img_rows, img_cols, 3) )


for (i,layer) in enumerate(model.layers):
    print(str(i)+" "+layer.__class__.__name__," ",layer.trainable)

Using TensorFlow backend.


0 InputLayer   False
1 Conv2D   True
2 Conv2D   True
3 MaxPooling2D   True
4 Conv2D   True
5 Conv2D   True
6 MaxPooling2D   True
7 Conv2D   True
8 Conv2D   True
9 Conv2D   True
10 MaxPooling2D   True
11 Conv2D   True
12 Conv2D   True
13 Conv2D   True
14 MaxPooling2D   True
15 Conv2D   True
16 Conv2D   True
17 Conv2D   True
18 MaxPooling2D   True


In [2]:
for layer in model.layers:
    layer.trainable = False
    
for (i,layer) in enumerate(model.layers):
    print(str(i)+" "+layer.__class__.__name__," ",layer.trainable)    

0 InputLayer   False
1 Conv2D   False
2 Conv2D   False
3 MaxPooling2D   False
4 Conv2D   False
5 Conv2D   False
6 MaxPooling2D   False
7 Conv2D   False
8 Conv2D   False
9 Conv2D   False
10 MaxPooling2D   False
11 Conv2D   False
12 Conv2D   False
13 Conv2D   False
14 MaxPooling2D   False
15 Conv2D   False
16 Conv2D   False
17 Conv2D   False
18 MaxPooling2D   False


In [3]:
def add_layers(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [4]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [5]:
model.layers

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 5
num_classes = 5

FC_Head = add_layers(model, num_classes)

model = Model(inputs = model.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [7]:
model.layers

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'train/'
validation_data_dir = 'test/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range = 45,
      width_shift_range = 0.3,
      height_shift_range = 0.3,
      horizontal_flip = True,
      fill_mode = 'nearest')

test_datagen = ImageDataGenerator(rescale=1./255)
 
batch_size = 32    
    
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_rows, img_cols),
        batch_size = batch_size,
        class_mode = 'categorical')
 
validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size = (img_rows, img_cols),
        batch_size = batch_size,
        class_mode = 'categorical')

Found 104 images belonging to 5 classes.
Found 13 images belonging to 5 classes.


In [9]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("celebrities.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

In [10]:
# Enter the number of training and validation samples here
nb_train_samples = 104
nb_validation_samples = 13

# We only train 10 EPOCHS 
epochs = 10
batch_size = 10

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/10
10/10 [==============================] - 76s 8s/step - loss: 1.9953 - accuracy: 0.1544 - val_loss: 1.6648 - val_accuracy: 0.1538

Epoch 00001: val_loss improved from inf to 1.66485, saving model to celebrities.h5
Epoch 2/10
10/10 [==============================] - 65s 7s/step - loss: 1.6089 - accuracy: 0.2581 - val_loss: 1.6397 - val_accuracy: 0.1538

Epoch 00002: val_loss improved from 1.66485 to 1.63965, saving model to celebrities.h5
Epoch 3/10
10/10 [==============================] - 59s 6s/step - loss: 1.5372 - accuracy: 0.2782 - val_loss: 1.5223 - val_accuracy: 0.3077

Epoch 00003: val_loss improved from 1.63965 to 1.52228, saving model to celebrities.h5
Epoch 4/10
10/10 [==============================] - 65s 7s/step - loss: 1.4535 - accuracy: 0.3493 - val_loss: 1.3626 - val_accuracy: 0.5385

Epoch 00004: val_loss improved from 1.52228 to 1.36257, saving model to celebrities.h5
Epoch 5/10
10/10 [==============================] - 65s 7s/step - loss: 1.5221 - accuracy: 0

In [11]:
from keras.models import load_model
classifier = load_model('celebrities.h5')

In [12]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

celebrities_dict = {"[0]": 'ben_afflek', 
                    "[1]": 'elton_john',
                    "[2]": 'jerry_seinfeld',
                    "[3]": 'madonna',
                    "[4]": 'mindy_kaling'}

celebrities_dict_n = {"n0": 'ben_afflek', 
                      "n1": 'elton_john',
                      "n2": 'jerry_seinfeld',
                      "n3": 'madonna',
                      "n4": 'mindy_kaling'}

def draw_test(name, pred, im):
    celebrity = celebrities_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, celebrity, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print(path_class)
    #print("Class - " + celebrities_dict_n[path_class])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

In [17]:
for i in range(0,3):
    input_im = getRandomImage("test/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)
cv2.destroyAllWindows()

mindy_kaling
ben_afflek
ben_afflek
